# AppenCorrect vLLM Testing on SageMaker

**Instance Required:** ml.g5.xlarge (NVIDIA A10G GPU, 24GB VRAM)

This notebook:
1. Installs vLLM + Flash Attention 2
2. Starts vLLM server with Qwen 2.5 7B
3. Starts Flask API
4. Creates ngrok tunnel for public access
5. Tests the complete system

**Time:** ~15 min first run (downloads 14GB model), then 2 min

## Step 1: Check GPU

In [ ]:
!nvidia-smi

## Step 2: Install Flash Attention 2

In [ ]:
print('Installing Flash Attention 2 (takes 5-10 min)...')
!pip install flash-attn==2.6.3 --no-build-isolation
print('\n✅ Flash Attention 2 installed')

## Step 3: Install vLLM and Dependencies

In [ ]:
!pip install vllm==0.6.3 transformers torch pyngrok requests flask flask-cors python-dotenv jsonschema langdetect
print('\n✅ All dependencies installed')

## Step 4: Navigate to Repository

In [ ]:
import os

# Navigate to your cloned repo
os.chdir('/home/sagemaker-user/appen-correct-localised')
!git checkout vllm
!git pull origin vllm

print(f'\n✅ Repository ready: {os.getcwd()}')

## Step 5: Start vLLM Server (Background)

In [ ]:
import subprocess, time, requests, os

# Set cache
cache = '/home/sagemaker-user/.huggingface'
os.makedirs(cache, exist_ok=True)
os.environ['HF_HOME'] = cache

# Kill existing
!pkill -f vllm.entrypoints || true
time.sleep(2)

print('🚀 Starting vLLM server...')
print('⏳ First run: 5-10 min (downloads 14GB)')
print('⏳ Next runs: 30-60 sec (from cache)\n')

vllm_process = subprocess.Popen([
    'python', '-m', 'vllm.entrypoints.openai.api_server',
    '--model', 'Qwen/Qwen2.5-7B-Instruct',
    '--host', '0.0.0.0', '--port', '8000',
    '--dtype', 'auto',
    '--max-model-len', '4096',
    '--gpu-memory-utilization', '0.85',
    '--max-num-seqs', '64',
    '--enable-prefix-caching'
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

for i in range(120):
    try:
        if requests.get('http://localhost:8000/health', timeout=2).status_code == 200:
            print(f'\n✅ vLLM ready after {i*5}s!')
            break
    except:
        if i % 6 == 0: print(f'  Loading... ({i*5}s)')
        time.sleep(5)

print('vLLM running at http://localhost:8000')

## Step 6: Test vLLM Directly

In [ ]:
import requests, json

r = requests.post('http://localhost:8000/v1/completions', json={
    'model': 'Qwen/Qwen2.5-7B-Instruct',
    'prompt': 'Fix: I has a eror',
    'max_tokens': 100, 'temperature': 0.2
}, timeout=30)

print('✅ vLLM inference test:')
print(r.json()['choices'][0]['text'])

## Step 7: Start Flask API

In [ ]:
import subprocess, time, requests, os

os.chdir('/home/sagemaker-user/appen-correct-localised')
os.environ['VLLM_URL'] = 'http://localhost:8000'

!pkill -f 'python.*app.py' || true
time.sleep(2)

print('🚀 Starting Flask API...')
flask_process = subprocess.Popen(
    ['python3', 'app.py'],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

time.sleep(5)
for i in range(10):
    try:
        r = requests.get('http://localhost:5006/health', timeout=2)
        if r.status_code == 200:
            print('\n✅ Flask API ready!')
            print(r.json())
            break
    except:
        time.sleep(2)

print('Flask running at http://localhost:5006')

## Step 8: Test Complete System

In [ ]:
r = requests.post('http://localhost:5006/demo/check', json={
    'text': 'I has a eror in grammer'
}, timeout=30)

result = r.json()
print('✅ Grammar check test:\n')
print(f'Original:  {result["original_text"]}')
print(f'Corrected: {result["corrected_text"]}')
print(f'\nErrors: {len(result["errors"])}')
for e in result['errors'][:3]:
    print(f'  {e["original"]} → {e["suggestion"]} ({e["type"]})')

## Step 9: Create ngrok Tunnel

**Get token:** https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
from pyngrok import ngrok, conf

NGROK_TOKEN = 'YOUR_TOKEN_HERE'  # ← CHANGE THIS!

if NGROK_TOKEN == 'YOUR_TOKEN_HERE':
    print('⚠️  Set your ngrok token above!')
    print('Get it: https://dashboard.ngrok.com/get-started/your-authtoken')
else:
    ngrok.kill()
    conf.get_default().auth_token = NGROK_TOKEN
    url = ngrok.connect(5006)
    
    print('='*70)
    print('🚀 PUBLIC URL:')
    print('='*70)
    print(f'\n{url}\n')
    print(f'Demo:   {url}/')
    print(f'Health: {url}/health')
    print(f'API:    {url}/demo/check')
    print('\n' + '='*70)
    print('\nShare this URL to test from anywhere!')

## Step 10: Get curl Command

In [ ]:
tunnels = ngrok.get_tunnels()
if tunnels:
    url = tunnels[0].public_url
    print('Copy this curl command:\n')
    print(f'curl -X POST {url}/demo/check \\\\')
    print('  -H "Content-Type: application/json" \\\\')
    print('  -d \'{"text": "I has a eror in grammer"}\'\n')
    print(f'Or open: {url}/')
else:
    print('Run ngrok cell first!')

## Monitor GPU

In [ ]:
!nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total --format=csv

## Cleanup

In [ ]:
!pkill -f 'python.*app.py' || true
!pkill -f vllm.entrypoints || true
from pyngrok import ngrok
ngrok.kill()
print('✅ Stopped all services')

---

## ✅ Done!

- vLLM server: Qwen 2.5 7B on GPU
- Flask API: Connected to vLLM
- ngrok: Public access
- Concurrency: 64 requests/GPU
- Model: Cached (no re-download)
- Cost: $0.75/hr vs $3k-5k/month API